In [ ]:
### Import Libraries.

import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scanpy as sc
import scvelo as scv
import cellrank as cr
from scipy import stats as st


scv.settings.verbosity = 3
scv.settings.set_figure_params(dpi = 800, figsize=(6,6))
sc.settings.set_figure_params(dpi = 800, figsize = (6,6))
sns.set_style("whitegrid")

In [ ]:
### Load Data.

os.chdir("/folder/")
adata = sc.read_h5ad("adata.h5ad")

In [ ]:
### CellRank: Transition Matrix Computation.

# Velocity kernel
vk = cr.kernels.VelocityKernel(adata)
vk.compute_transition_matrix(model = 'monte_carlo')

# Connectivity kernel
ck = cr.kernels.ConnectivityKernel(adata)
ck.compute_transition_matrix()

# Combine kernels
combined_kernel = 0.8 * vk + 0.2 * ck

In [ ]:
### Visualize Velocity Projections

vk.plot_projection(
    arrow_size = 2,
    basis = "umap",
    color = "Cluster_Column",
    density = 2
)

In [ ]:
### Random Walks from Selected Starting Cells.

start_cells = {
    "Cluster_1": "Start_Cell_1",
    "Cluster_2": "Start_Cell_2"
}
for cell_name, cluster_key in start_cells.items():
    vk.plot_random_walks(start_ixs = {cluster_key: cell_name}, basis = "umap", max_iter = 1000, seed = 0)

In [ ]:
### Velocity Pseudotime and Visualization.

scv.tl.velocity_pseudotime(adata)
sc.pl.embedding(
    adata,
    basis = "umap",
    color = "velocity_pseudotime",
    color_map = "gnuplot2",
    vmin = 0, vmax = 0.2,
    size = 30
)

In [ ]:
### Clip Pseudotime for Plot.

adata.obs["velocity_pseudotime_clipped"] = adata.obs["velocity_pseudotime"].clip(lower = 0, upper = 0.2) ### Cange upper

sc.pl.violin(
    adata,
    keys = ["velocity_pseudotime_clipped"],
    groupby = "Cluster_Column",
    rotation = -90,
    stripplot = False
)

In [ ]:
##Pseudotime Kernel. 

pk = cr.kernels.PseudotimeKernel(adata, time_key = "velocity_pseudotime")
pk.compute_transition_matrix()
pk.plot_projection(basis = "umap", recompute = True)

In [ ]:
### GPCCA: Initial and Terminal States.

gpcca = cr.estimators.GPCCA(vk)
gpcca.fit(cluster_key = "Cluster_Column", n_states = [4,12])

In [ ]:
### Macrostates.

gpcca.plot_macrostates(which="all", basis = "umap", discrete = True, legend_loc = "right", s = 100)
gpcca.predict_terminal_states()
gpcca.plot_macrostates(which = "terminal", basis = "umap", legend_loc = "right", s = 100)
gpcca.predict_initial_states(allow_overlap = True)
gpcca.plot_macrostates(which = "initial", basis = "umap", legend_loc = "right", s = 100)

In [ ]:
### Subset example: Beta cells

adata_sub = adata[adata.obs["Cluster_Column"] == "Cluster_1"].copy()

adata_sub.obs["maturation_state"] = np.where(
    adata_sub.obs["term_states_fwd"] == "Cluster_1",
    "terminal", "not terminal"
)
sc.pl.violin(adata_sub, keys = ["JUND"], groupby = "maturation_state")

In [ ]:
### Statistical test.

a = adata_sub[adata_sub.obs["maturation_state"] == "terminal", "JUND"].X.data
b = adata_sub[adata_sub.obs["maturation_state"] == "not terminal", "JUND"].X.data
t_stat, p_val = st.ttest_ind(a, b, equal_var = False)
print(f"T-test JUND (terminal vs non-terminal): t = {t_stat:.3f}, p = {p_val:.3e}")

In [ ]:
### (Optional) Manual Setting of Terminal States.

gpcca2.set_terminal_states(states = adata.obs["Cluster_Column"])
gpcca2.plot_macrostates(which = "terminal", basis = "umap", legend_loc = "right", s = 100)

#### Compute Fate Probabilities.

gpcca2.compute_fate_probabilities()
gpcca2.plot_fate_probabilities(basis = "umap", same_plot = False)
gpcca2.plot_fate_probabilities(basis = "umap", same_plot = True)

In [ ]:
### Circular projection visualization

cr.pl.circular_projection(
    adata,
    keys = ["Cluster_Column_1", "Cluster_Column_2"],
    legend_loc = "right"
)